In [42]:
import pandas as pd
from urllib import urlopen
from bs4 import BeautifulSoup
from bs4 import Comment
import re
import json




In [43]:
## parameters and bs4 connection
url = "https://www.linestarapp.com/Ownership/Sport/WNBA/Site/FanDuel/PID/379"
output_path = '/output/path/goes/here/'

html = urlopen(url)
print(html)
soup = BeautifulSoup(html, "html.parser")

#print(soup)


<addinfourl at 4432183448 whose fp = <socket._fileobject object at 0x108147cd0>>


In [44]:
## scrape page to get pretty column headers
column_headers = [td.getText() for td in
                      soup.findAll('tr', limit=10)[1].findAll('td')]

print(column_headers)

[u'Player', u'Pos', u'Team', u'Salary', u'Actual Ownership']


In [45]:
## function definitions

def format_script_string(hold):
    hold = hold.replace('\r', '')
    hold = hold.replace('\n', '')
    return(hold)

def format_js_var_string(x):
    x = x.replace(sub_str, '').replace("'", '"')
    x = x.replace('            ', ' ').replace(';', '') ## replacing the big space isn't needed but makes the string easier to read
    ## these are specific to the js variable dict... must be adjusted based on what is pulled  
    ## want to replace each key so that json.loads can read it correctly
    ## since each key is a string need to add quotes to it to make it {"key":value} format
    x = x.replace('id:', '"id":') 
    x = x.replace('name:', '"name":')
    x = x.replace('owned:', '"owned":')
    x = x.replace('pos:', '"pos":')
    x = x.replace('team:', '"team":')
    x = x.replace('sal:', '"sal":')
    ## finally, we replace the pesky piece of the js variable.. the fucking ',}' and ',]' (these don't work w/ json loads)
    x = re.sub(',\s*}', '}', x) ## note: this give weird results depending on the strings passed.. 
    x = re.sub(',\s*]', ']', x)
    return(x)
            
            
def scrape_scripts(sub_str, scripts):
    #html_scripts = soup.findAll('script')
    #sub_str = 'var actualResultsDict = '
    res = {}
    temp = re.compile(sub_str + '{(.*)};', re.MULTILINE) 
    for script in scripts: ## loop thru scripts tags, then search the strings for matches of the `temp` search above
        hold = format_script_string(str(script.string))
        x = temp.search(hold) ## do the search 
        if x: ## if we found something then.. 
            x = x.group()
            x = format_js_var_string(x) ## format js variable string 
            res = json.loads(x)
    return(res)



In [46]:
sub_str = 'var actualResultsDict = '  ## this is hte js variable dict that holds the table data 
#sub_str = 'var projectedSlatesDict = ' ## this is hte js variable dict that holds the table data 

json_object = scrape_scripts(sub_str, soup.findAll('script'))

## need to format the json_object a bit for pandas
res = []
for slate_id, sub_list in json_object.items():
    for val in sub_list:
        val.update({'slate_id':slate_id})
        res.append(val)

        
res = pd.DataFrame(res)


In [47]:
## rename pandas df 
new_names = {'id': 'player_id', ## could be wrong about this.. not really sure what the ID is.. 
             'owned': 'ownership',
             'pos': 'position',
             'sal': 'salary',
             'team':'team',
             'slate_id': 'slate_id'  ## have no clue what this is... take a look at print(soup) and youll see how the variables are structured
            }

res.rename(columns = new_names, inplace=True)

In [48]:
res

,player_id,name,ownership,position,salary,slate_id,team
0,224,Astou Ndour,59.2,F,3000,0,CHI
1,127,Kayla McBride,55.2,G,4700,0,LV
2,76,LaToya Sanders,34.6,F,3900,0,WAS
3,15,Cheyenne Parker,33.7,F,4800,0,CHI
4,145,Breanna Stewart,32.8,F,8500,0,SEA
5,70,Elena Delle Donne,26.4,F,8300,0,WAS
6,84,Skylar Diggins-Smith,25.6,G,8000,0,DAL
7,72,Kristi Toliver,24.7,G,5000,0,WAS
8,104,Sylvia Fowles,24.5,F,8300,0,MIN
9,360,Diamond DeShields,24.4,G,4500,0,CHI


In [ ]:
#res.to_csv(output_path + 'res.csv')